In [11]:
from datasets import load_dataset
import pandas as pd 
from sklearn.model_selection import train_test_split

In [12]:
from datasets import load_dataset

# Load directly from Hugging Face Hub by its dataset identifier
hate_explain = load_dataset("Abhi0072/HateXplain")

# convert to dataframe
hate_explain = hate_explain['train'].to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


In [13]:
hate_explain.info()
hate_explain.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20148 entries, 0 to 20147
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20148 non-null  object
 1   label   20148 non-null  object
dtypes: object(2)
memory usage: 314.9+ KB


,text,label
0,i dont think im getting my baby them white 9 h...,normal
1,we cannot continue calling ourselves feminists...,normal
2,nawt yall niggers ignoring me,normal
3,<user> i am bit confused coz chinese ppl can n...,hatespeech
4,this bitch in whataburger eating a burger with...,hatespeech
5,laura loomer raped me while screaming at me in...,hatespeech
6,and this is why i end up with nigger trainee d...,hatespeech
7,nogs jews and dykes how enriching,offensive
8,guilty until proven innocent unless you are a ...,hatespeech
9,tired i can not support abortion from a moral ...,hatespeech


In [14]:
label_mapping = {
    "normal": 2,
    "hatespeech": 0,
    "offensive": 1

}

# Apply the mapping to the label column
hate_explain['label'] = hate_explain['label'].apply(lambda label: label_mapping.get(label))

In [15]:
hate_explain.head(10)

,text,label
0,i dont think im getting my baby them white 9 h...,2
1,we cannot continue calling ourselves feminists...,2
2,nawt yall niggers ignoring me,2
3,<user> i am bit confused coz chinese ppl can n...,0
4,this bitch in whataburger eating a burger with...,0
5,laura loomer raped me while screaming at me in...,0
6,and this is why i end up with nigger trainee d...,0
7,nogs jews and dykes how enriching,1
8,guilty until proven innocent unless you are a ...,0
9,tired i can not support abortion from a moral ...,0


In [16]:
davidson_dataset = load_dataset("tdavidson/hate_speech_offensive")
# convert to dataframe
davidson_df = davidson_dataset['train'].to_pandas()

In [17]:
# Rename 'class' column to 'label'
davidson_df = davidson_df.rename(columns={'class': 'label', 'tweet': 'text'})
davidson_df = davidson_df.drop(columns=["neither_count", "offensive_language_count", "hate_speech_count","count"])

In [18]:
davidson_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   24783 non-null  int64 
 1   text    24783 non-null  object
dtypes: int64(1), object(1)
memory usage: 387.4+ KB


In [19]:
gab = load_dataset("juliadollis/The_Gab_Hate_Corpus_ghc_train_original")
# convert to dataframe
gab = gab['train'].to_pandas()

# Filter hate speech (either attacks dignity or calls for violence)
gab_hate = gab[(gab["hd"] == 1) | (gab["cv"] == 1)].copy()

gab_hate['label'] = 0
gab_hate = gab_hate.drop(columns=["hd", "cv", "vo"])


# Filter only offensive speech (vulgar but not hateful)
gab_offensive = gab[(gab["vo"] == 1) & (gab["hd"] == 0) & (gab["cv"] == 0)].copy()

gab_offensive['label'] = 1
gab_offensive = gab_offensive.drop(columns=["hd", "cv", "vo"])
gab_offensive.head()

gab = pd.concat([gab_hate, gab_offensive])

gab.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2670 entries, 29 to 21997
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2670 non-null   object
 1   label   2670 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 62.6+ KB


In [20]:
frankshu = load_dataset("thefrankhsu/hate_speech_twitter")
# convert to dataframe
frankshu = frankshu['train'].to_pandas()

frankshu_hate = frankshu[frankshu['label'] == 1].copy()
frankshu_hate['label'] = 0

frankshu_normal = frankshu[frankshu['label'] == 0].copy()
frankshu_normal['label'] = 2

frankshu = pd.concat([frankshu_hate, frankshu_normal])

frankshu = frankshu.drop(columns=['categories'])
frankshu = frankshu.rename(columns={"tweet": "text"})
frankshu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5679 entries, 1 to 5678
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5678 non-null   object
 1   label   5679 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.1+ KB


In [ ]:
frankshu = load_dataset("")

In [21]:
dataset = pd.concat([hate_explain, davidson_df, gab, frankshu])

In [22]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53280 entries, 0 to 5678
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    53279 non-null  object
 1   label   53280 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [23]:
dataset["label"].value_counts()

label
1    25867
2    16535
0    10878
Name: count, dtype: int64

In [24]:
print(dataset.iloc[:10]['text'])

0    i dont think im getting my baby them white 9 h...
1    we cannot continue calling ourselves feminists...
2                        nawt yall niggers ignoring me
3    <user> i am bit confused coz chinese ppl can n...
4    this bitch in whataburger eating a burger with...
5    laura loomer raped me while screaming at me in...
6    and this is why i end up with nigger trainee d...
7                    nogs jews and dykes how enriching
8    guilty until proven innocent unless you are a ...
9    tired i can not support abortion from a moral ...
Name: text, dtype: object


In [31]:
train_data = pd.read_csv("../data/processed_training_data.csv")

In [32]:
train_data['label'].value_counts()

label
1    18117
2    11581
0     7619
Name: count, dtype: int64